In [6]:
# """ combine query predictions and output as a result json file """
# import os
# from utils import execute_all
# path1 = "/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions_0.json"
# path2 = "/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions_1.json"

# # combine two json files
# import json
# with open(path1) as f:
#     data1 = json.load(f)
# with open(path2) as f:
#     data2 = json.load(f)

# data1.update(data2)

# DB_PATH = os.path.join('data', 'mimic_iv', 'mimic_iv.sqlite')
# predictions = execute_all(data1, db_path=DB_PATH, tag='pred')

# with open("/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions.json", "w") as f:
#     json.dump(predictions, f, indent=4)

### Hard Vote Ensemble

In [1]:
import json

PATHS = [  
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold2/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold3/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold4/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold5/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large/predictions.json",
  "./RESULTS/baselinev4-flan-t5-text2sql-with-schema-v2/predictions.json",
]

In [2]:
import pandas as pd

df = pd.DataFrame(columns=["id"])
for idx, json_path in enumerate(PATHS):
  column_name = f"answer_{idx}"
  df[column_name] = None
  with open(json_path, "r") as f:
    data = json.load(f)
    if idx == 0: # initialize column
      df["id"] = list(data.keys())
    for key in data: # append data
      df.loc[df["id"] == key, column_name] = data[key]

df.head(10)

,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7
0,d084d1f3c277e6827087bb44,null,null,null,null,null,null,null,null
1,a35a9346ab483d0db0f202ca,[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']]
2,144cd6f1acfad4416003c26c,"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...",[],"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...",[],"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ..."
3,b20d40188481222bfbb9b02f,[['2100-09-11 14:48:00']],[['2100-09-06 23:09:00']],[['ew emer.']],error_pred,error_pred,[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']]
4,f5f185ff5f7901dc7c4dd711,[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']]
5,e6db613772003ec72d44ebe5,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...
6,769ea1c5d6c42c47ac9a1735,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]
7,6730aa47b18b0105eb3dd8a2,"[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]"
8,9d0210cd4045e7f8e860ce69,error_pred,null,null,null,null,null,null,null
9,ffc47b7e01463f229eb09bce,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]


In [7]:
# find rows with different answers
diff_row_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if len(set(answers)) > 1:
    diff_row_idxs.append(idx)

df_diff = df.loc[diff_row_idxs]
print(f"Number of different rows: {len(df_diff)}")
df_diff.head(10)

Number of different rows: 438


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7
2,144cd6f1acfad4416003c26c,"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...",[],"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...",[],"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ..."
3,b20d40188481222bfbb9b02f,[['2100-09-11 14:48:00']],[['2100-09-06 23:09:00']],[['ew emer.']],error_pred,error_pred,[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']]
8,9d0210cd4045e7f8e860ce69,error_pred,null,null,null,null,null,null,null
10,2afd67ca4b6eb596928b1024,error_pred,error_pred,[['0.0']],[['0.0']],[['0.0']],[['0.0']],error_pred,null
11,dab078420665e61aa6883a1d,[['1.0']],[['1.0']],[['0.0']],[['1.0']],[['1.0']],[['1.0']],[['1.0']],[['1.0']]
12,10f5ecdf9123785c95f2bff6,"[['blood culture, routine'], ['c. difficile pc...",error_pred,[],[],"[['blood culture, routine'], ['c. difficile pc...","[['blood culture, routine'], ['c. difficile pc...",error_pred,[]
15,be6fb8ffe49a397f297313f4,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...",error_pred,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['cerebral ventricular #1'], ['chest tube #1'...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9..."
16,96fb52e3ec9186d401d3bfb1,error_pred,null,null,null,[],null,null,null
24,97abc537c029974692f9c1af,null,null,null,null,null,null,null,[['0.0']]
27,04987dfc639773e714441af1,[],"[['0.9% sodium chloride'], ['5% dextrose'], ['...","[['apixaban'], ['bag'], ['insulin'], ['insulin...",error_pred,"[['apixaban'], ['bag'], ['insulin'], ['insulin...","[['0.9% sodium chloride'], ['bag'], ['furosemi...",null,"[['0.9% sodium chloride'], ['bag'], ['furosemi..."


In [8]:
# iterrate over rows and check error_pred
error_pred_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers:
    error_pred_idxs.append(idx)

df_error_pred = df.loc[error_pred_idxs]
print(f"Number of error_pred rows: {len(df_error_pred)}")
df_error_pred.head(10)

Number of error_pred rows: 281


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7
3,b20d40188481222bfbb9b02f,[['2100-09-11 14:48:00']],[['2100-09-06 23:09:00']],[['ew emer.']],error_pred,error_pred,[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']]
8,9d0210cd4045e7f8e860ce69,error_pred,null,null,null,null,null,null,null
10,2afd67ca4b6eb596928b1024,error_pred,error_pred,[['0.0']],[['0.0']],[['0.0']],[['0.0']],error_pred,null
12,10f5ecdf9123785c95f2bff6,"[['blood culture, routine'], ['c. difficile pc...",error_pred,[],[],"[['blood culture, routine'], ['c. difficile pc...","[['blood culture, routine'], ['c. difficile pc...",error_pred,[]
15,be6fb8ffe49a397f297313f4,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...",error_pred,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['cerebral ventricular #1'], ['chest tube #1'...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9..."
16,96fb52e3ec9186d401d3bfb1,error_pred,null,null,null,[],null,null,null
27,04987dfc639773e714441af1,[],"[['0.9% sodium chloride'], ['5% dextrose'], ['...","[['apixaban'], ['bag'], ['insulin'], ['insulin...",error_pred,"[['apixaban'], ['bag'], ['insulin'], ['insulin...","[['0.9% sodium chloride'], ['bag'], ['furosemi...",null,"[['0.9% sodium chloride'], ['bag'], ['furosemi..."
31,4f51bf550b78551e0824cb6c,error_pred,error_pred,error_pred,error_pred,error_pred,error_pred,error_pred,null
33,44162aee19eabadbab9a0172,error_pred,error_pred,error_pred,error_pred,error_pred,error_pred,error_pred,null
37,fcb5ab40c54f3bce352480bc,[['2100-09-27 17:09:00']],[['2100-05-07 13:50:00']],[['eu observation']],error_pred,[['2100-03-23 15:34:00']],[['2100-03-23 15:34:00']],[['2100-03-23 15:34:00']],null


In [5]:
# write df as new json file
ensembled_dict = {}
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers: # pass the difficult question
    ensembled_dict[row["id"]] = "null"
    continue
  if len(set(answers)) == 1:
    ensembled_dict[row["id"]] = answers[0]
  else:
    ensembled_dict[row["id"]] = "null"

In [6]:
import json

%cd ./RESULTS/ENSEMBLE
path = "./ensemble_v4_six.json"
zip_path = path.replace(".json", ".zip")
with open(path, "w") as f:
    json.dump(ensembled_dict, f, indent=4)

!zip $zip_path $path

/ext_hdd/yjahn/joint-learning/RESULTS/ENSEMBLE
updating: ensemble_v4_six.json (deflated 73%)


/home/yjahn/miniconda3/envs/ehr/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
